In [18]:
import random

professors = {
    'Neto': [(0, 19), (0, 20), (0, 21),  # Segunda-feira, noite
             (1, 18), (1, 21),  # Terça-feira, noite
             (2, 18), (2, 19), (2, 20), (2, 21),  # Quarta-feira, noite
             (4, 18), (4, 19), (4, 20), (4, 21)], # Sexta-feira, noite

    'Chiquinho': [(0, 18), (0, 19), (0, 20),  # Segunda-feira, noite
                  (1, 18), (1, 19), (1, 20), (1, 21),  # Terça-feira, noite
                  (2, 20), (2, 21)],

    'Maria': [(0, 19), (0, 20),
              (2, 19), (2, 20), (2, 21),
              (3, 21),
              (4, 18), (4, 19), (4, 21)],

    'Joao': [(0, 18), (0, 19),
             (1, 20), (1, 21)],
}


# Matéria, professor e carga horária em horas
subjects = {
    'Redes': ('Neto', 72),
    'Logica': ('Chiquinho', 36),
    'Matematica': ('Maria', 36),
    'Fisica': ('Joao', 72),
}


classRooms = {
    'Sala 10': [(0, 18), (0, 19), (0, 20), (0, 21),
              (1, 18), (1, 19), (1, 20), (1, 21),
              (2, 18), (2, 19), (2, 20), (2, 21),
              (3, 18), (3, 19), (3, 20), (3, 21),
              (4, 18), (4, 19), (4, 20), (4, 21)]
}


POPULATION_SIZE = 100
MUTATION_RATE = 0.1
CROSSOVER_RATE = 0.8
GENERATIONS = 1000
FITNESS_THRESHOLD = 0.9
NUM_CLASSES = sum((72 // 1.5 if hours == 72 else 36 // 1.5) for subject, (professor, hours) in subjects.items()) * len(rooms)


In [19]:
# Gerando a população inicial
def generate_initial_population():
    population = []
    for _ in range(POPULATION_SIZE):
        chromosome = []
        for subject, (professor, hours) in subjects.items():
            # Qtd de aulas por semana
            if hours == 72:
                sessions_per_week = 2  # 2 por semana
            elif hours == 36:
                sessions_per_week = 1  # 1 por semana

            for room in classRooms:
                for _ in range(sessions_per_week):
                    available_times = list(set(professors[professor]) & set(classRooms[room]))
                    day, time = random.choice(available_times)
                    chromosome.append((subject, room, day, time))
        population.append(chromosome)
    return population


# Fitness de um cromossomo
def calculate_fitness(chromosome):
    fitness = 1.0
    conflicts = 0

    timeslot_count = {}
    room_usage = {}

    for subject, room, day, time in chromosome:
        # Contar uso de horários por professor
        if (subjects[subject][0], day, time) in timeslot_count:
            timeslot_count[(subjects[subject][0], day, time)] += 1
        else:
            timeslot_count[(subjects[subject][0], day, time)] = 1

        # Contar uso de salas
        if (room, day, time) in room_usage:
            room_usage[(room, day, time)] += 1
        else:
            room_usage[(room, day, time)] = 1

    for count in timeslot_count.values():
        if count > 1:
            conflicts += (count - 1)

    for count in room_usage.values():
        if count > 1:
            conflicts += (count - 1)

    # Reduzir a fitness por conflito
    fitness -= conflicts * 0.1

    if fitness < 0:
        fitness = 0

    return fitness

def selection(population):
    mating_pool = []
    population_fitness = [(chromosome, calculate_fitness(chromosome)) for chromosome in population]
    population_fitness.sort(key=lambda x: x[1], reverse=True)

    for i in range(len(population_fitness)):
        if random.random() < (i / len(population_fitness)):
            mating_pool.append(population_fitness[i][0])
    return mating_pool

# Realizando crossover entre os cromossomos do pool de acasalamento
def crossover(mating_pool):
    offspring = []
    for _ in range(len(mating_pool) // 2):
        parent1 = random.choice(mating_pool)
        parent2 = random.choice(mating_pool)
        if random.random() < CROSSOVER_RATE:
            point = random.randint(1, len(parent1) - 1)
            child1 = parent1[:point] + parent2[point:]
            child2 = parent2[:point] + parent1[point:]
            offspring.extend([child1, child2])
        else:
            offspring.extend([parent1, parent2])
    return offspring

def mutate(mating_pool):
    for chromosome in mating_pool:
        if random.random() < MUTATION_RATE:
            index = random.randint(0, len(chromosome) - 1)
            subject, room, _, _ = chromosome[index]
            available_times = list(set(professors[subjects[subject][0]]) & set(classRooms[room]))
            day, time = random.choice(available_times)
            chromosome[index] = (subject, room, day, time)
    return mating_pool

# Transferindo cromossomos melhores para a população
def replace_population(population, mating_pool):
    population_fitness = [(chromosome, calculate_fitness(chromosome)) for chromosome in population]
    population_fitness.sort(key=lambda x: x[1])

    mating_pool_fitness = [(chromosome, calculate_fitness(chromosome)) for chromosome in mating_pool]
    mating_pool_fitness.sort(key=lambda x: x[1], reverse=True)

    for i in range(len(mating_pool_fitness)):
        if mating_pool_fitness[i][1] > population_fitness[i][1]:
            population_fitness[i] = mating_pool_fitness[i]

    return [chromosome for chromosome, fitness in population_fitness]

def genetic_algorithm():
    population = generate_initial_population()
    for generation in range(GENERATIONS):
        # Calcula a "nota" (fitness) de cada conjunto de horários
        population_fitness = [calculate_fitness(chromosome) for chromosome in population]

        # Encontra a melhor "nota" atual
        best_fitness = max(population_fitness)

        if best_fitness >= FITNESS_THRESHOLD:
            best_chromosome = population[population_fitness.index(best_fitness)]
            print(f"Melhor conjunto de horários encontrado na geração {generation}: {best_chromosome} com nota {best_fitness}")
            return best_chromosome

        mating_pool = selection(population)
        mating_pool = crossover(mating_pool)
        mating_pool = mutate(mating_pool)
        population = replace_population(population, mating_pool)

    best_chromosome = population[population_fitness.index(max(population_fitness))]
    print(f"Melhor conjunto de horários encontrado após {GENERATIONS} gerações: {best_chromosome} com nota {max(population_fitness)}")
    return best_chromosome


best_timetable = genetic_algorithm()

Melhor conjunto de horários encontrado na geração 0: [('Redes', 'Sala 10', 4, 19), ('Redes', 'Sala 10', 0, 21), ('Logica', 'Sala 10', 1, 18), ('Matematica', 'Sala 10', 2, 19), ('Fisica', 'Sala 10', 0, 19), ('Fisica', 'Sala 10', 1, 20)] com nota 1.0
